# Создание новых признаков для таблиц

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys, os
import pathlib
from pathlib import Path
from pydantic import ValidationError
from olist_churn_prediction.paths import SRC_DIR, PROCESSED_DIR, INTERIM_DIR, RAW_DIR
from olist_churn_prediction import feature_processing, feature_engineering

## classified_data_interim

In [ ]:
from olist_churn_prediction.schemas_interim import MainClassifiedSchemaInterim

classified_data = feature_processing.load_data(INTERIM_DIR / "classified_data_interim.parquet",
                      schema = MainClassifiedSchemaInterim,
                      validate = True) # в проде можно False

### Создание временных признаков

#### На основе временных признаков можно создать новые полезные и хорошо интерпретируемые:
1. Время между оплатой заказа и подтверждением заказа на сайте (order_approved_at - order_purchase_timestamp).
2. Время между подтверждением заказа и ориентировочным времени доставки (order_estimated_delivery_date - order_aproved_at).
3. Время между фактическим времени доставки и ожидаемым (order_delivered_customer_date - order_estimated_delivery_date).
4. Время между рассылкой с просьбой оценить качество и ответом покупателя (review_answer_timestamp - review_creation_date).
5. Время между подтверждением заказа и его прибытием (order_delivered_customer_date - order_aproved_at).

In [ ]:
def new_time_features(data, feature_1, feature_2):
    return (data[feature_2] - data[feature_1]).dt.total_seconds() / 3600

In [ ]:
df['aproved-purchased'] = new_time_features(df, 'order_purchase_timestamp', 'order_aproved_at')
df['estimated_delivery-aproved'] = new_time_features(df, 'order_aproved_at', 'order_estimated_delivery_date')
df['actual_delivery-estimated_delivery'] = new_time_features(df, 'order_estimated_delivery_date', 'order_delivered_customer_date')
df['review_answer-review_creation'] = new_time_features(df, 'review_creation_date', 'review_answer_timestamp')
df['actual_delivery-aproved'] = new_time_features(df, 'order_aproved_at', 'order_delivered_customer_date')

In [ ]:
display(df[['aproved-purchased', 'estimated_delivery-aproved', 'actual_delivery-estimated_delivery', 'review_answer-review_creation', 'actual_delivery-aproved']].head())

#### Значения показывают количество часов разницы. В случае с разницей в доставке число отрицательное если заказ был доставлен раньше ориентировочного числа (скорее хорошо), и положительное, если была задержка по сравнению с ориентировочным числом (скорее плохо).

## Новые классы из votes-признаков

In [ ]:
# max_votes: максимальное число голосов в строке
# ties_mask: True там, где столбец == max_votes
max_votes  = df[votes_features].max(axis=1)
ties_mask  = df[votes_features].eq(max_votes, axis=0) # широкая матрица True/False такой же формы

display(max_votes.head())
display(ties_mask.head())

In [ ]:
# формируем новые индикаторы
new_cols = [c.replace('votes_', '') for c in votes_features] # ['satisfied', 'partial_delivery', 'not_as_anounced', ...]
df[new_cols] = ties_mask.astype(int).values # True→1, False→0

In [ ]:
display(df[new_cols].head())